<a href="https://colab.research.google.com/github/Cnava8554/Big-Data-ETL/blob/main/part_one_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:14 http://sec

In [11]:
print(os.environ["SPARK_HOME"])

/content/spark-3.3.2-bin-hadoop2.7


In [10]:
"C:\Users\christian.nava\Downloads\spark-3.3.2-bin-hadoop3.tgz"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-16 00:53:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.46MB/s    in 0.2s    

2023-03-16 00:53:34 (5.46 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/big-data-etl-3.13.2023/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True)

df = df.drop_duplicates()

df = df.dropna()

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49604665|R101MKYIB93PM2|B002RT7K3W|     813282509|DEWALT Bare-Tool ...|           Tools|          5|            0|          1|   N|                Y|    XRP Hammer drill|Very good price. ...| 2012-08-28|
|         US|   10750190|R1024JJSPZMP88|B007XGWQ2Y|     173533551|iFixit Pro Tech T...|           Tools|          5|    

In [6]:
# Get the number of rows in the DataFrame.
df.count()

1740974

# Transform the Data

## Create the "review_id_table".

In [10]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))

review_id = review_id.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))

review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R101MKYIB93PM2|   49604665|B002RT7K3W|     813282509| 2012-08-28|
|R1024JJSPZMP88|   10750190|B007XGWQ2Y|     173533551| 2014-07-28|
|R102GVOTB8CGWR|   14908188|B00I04J2OI|     888761841| 2015-07-12|
|R102RQU98Y6TLD|   25453680|B0011E6H88|     146248232| 2013-03-12|
|R1042GFR77Z4AB|   12334123|B00KQU1ENG|     809195352| 2014-11-03|
|R104BTKISM889K|   26633734|B004FWHOVC|     977333517| 2013-11-05|
|R104PBEN7LFH03|   12026456|B003D3N7WO|     175335407| 2014-03-01|
|R105EQ5WO5V47A|   51184875|B00006JBL3|     430796809| 2012-06-19|
|R105Z5FCA8G3NL|   16917983|B00CHJHA8W|     767610957| 2014-11-07|
|R1062KHF4BHKAP|   23175152|B001M0O170|     735138046| 2015-02-16|
|R107NZ3UKAR5RG|   11737996|B00F9Y1OS4|     662388793| 2014-07-09|
|R107UALI3WGGXA|   13132834|B003EIKBUY|     175271596| 2012-05

## Create the "products" Table

In [11]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
from pyspark.sql.functions import to_date, col, substring 
products = df.select("product_id", substring(col("product_title"), 1,300).alias("product_title"))

# Drop duplicates
products = products.drop_duplicates()

# Show the resulting DataFrame
products.show()



+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [12]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product.  
customers = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))

customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   19697083|            26|
|   40354137|             2|
|    7053720|             1|
|   50429720|            92|
|   35729428|             7|
|   13021566|             2|
|   21837304|             1|
|   28785149|             3|
|   36663024|             1|
|    6699560|             2|
|   17967296|             1|
|   48415930|             2|
|   10828210|             1|
|   46861820|             1|
|   10216459|             1|
|   41489840|             1|
|   23290046|            12|
|   15977325|             8|
|    8297781|             1|
|   29910871|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [13]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))

vine.show()



+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101MKYIB93PM2|          5|            0|          1|   N|
|R1024JJSPZMP88|          5|            1|          1|   N|
|R102GVOTB8CGWR|          5|            0|          0|   N|
|R102RQU98Y6TLD|          5|            0|          0|   N|
|R1042GFR77Z4AB|          5|            3|          4|   N|
|R104BTKISM889K|          3|            0|          0|   N|
|R104PBEN7LFH03|          5|            0|          0|   N|
|R105EQ5WO5V47A|          5|            0|          0|   N|
|R105Z5FCA8G3NL|          4|            0|          0|   N|
|R1062KHF4BHKAP|          5|            0|          0|   N|
|R107NZ3UKAR5RG|          5|            0|          0|   N|
|R107UALI3WGGXA|          5|            3|          3|   N|
|R108SUZQ6L9M5J|          5|            1|          1|   N|
|R10AAOQARC61CW|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
